In [2]:
## Building a motion detector
## This application will be able to detect and localize movement in a video feed.
# Preprocessing
import cv2
# Reading the video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read() # ret = return value is a boolean variable
    # if return value is True
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # In opencv defaultreading format = bgr
    
    if ret:
        blurred = cv2.GaussianBlur(frame, (21,21), 0)
        ## Applying threshold to the image
        ret, thresh = cv2.threshold(frame, 20, 255, cv2.THRESH_BINARY) # above 20 pixel value = white
        dilated = cv2.dilate(frame, None, iterations = 2)
        
        cv2.imshow('Threshold', thresh)
        cv2.imshow('dilated', dilated)
        cv2.imshow('Blurred', blurred)
        # cv2.imshow('HSV', hsv)
        cv2.imshow('Demo', frame) # every video can be broken down into a series of images.
    else:
        break
    k = cv2.waitKey(10)
    if k == ord('q'):
        cap.release()
        break  
cv2.destroyAllWindows()          

In [1]:
## Consider two back to back frames A and B
# Computing the mathematical diff bw them will produce a third image highlighting the changes from A to B
# If this change exceeds a certain user set boundary we can conclude that a motion has occured
import cv2
import numpy as np
# Reading the video
cap = cv2.VideoCapture(0)

ret1, frame1 = cap.read() # ret = return value is a boolean variable
ret2, frame2 = cap.read()
while True:
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    blurred1 = cv2.GaussianBlur(frame1_gray, (21,21), 0)
    blurred2 = cv2.GaussianBlur(frame2_gray, (21,21), 0)
    
    diff = cv2.absdiff(blurred1, blurred2)
    thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    final = cv2.dilate(thresh, None, iterations = 2)
    masked = cv2.bitwise_and(frame1, frame1, mask = thresh)
    ## Applying masks to images
    ## Tigger actions based on the amount of motion detected 
    ## Its just bitwise AND of the original image and the mask..
    
    ## Tigger:
    # just count the no. of white pixels in an image(255)
    # If this number exceed 1% of the total pixels, we can tigger a user defined action..
    white_pixels = np.sum(thresh/255.)
    rows, cols = thresh.shape
    total = rows*cols ## Total pixes in an image
    if white_pixels > 0.01*total:
        ## The action is to write a text on the screen
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame1, 'Motion Detected', (10, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow('Motion', frame1)
    
    frame1 = frame2
    ret, frame2 = cap.read()
    ## Only when there will a motion in the picture you will white pixels
    
    if not ret:
        break
        
    k = cv2.waitKey(10)
    if k == ord('q') or k == 27:
        break

cv2.destroyAllWindows()    